# Initialization

In [1]:
import time
import numpy as np
import pandas as pd
import qgrid

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

from poloniex import Poloniex
polo = Poloniex()

# Utility functions

## Load trade data from Poloniex

In [2]:
def loadPolo(pair, period, start, end):
    dates = []
    data={'price':[],'volume':[]}
    raw = polo.returnChartData(currencyPair=pair,period=period,start=start,end=end)
    for i in range(len(raw)):
        dates.append(int(raw[i]['date']))
        data['price'].append( float(raw[i]['weightedAverage']))
        data['volume'].append( float(raw[i]['volume'] ))
    return pd.DataFrame( data, index=pd.to_datetime(dates,unit='s'))

## Weighted moving average and MACD

In [3]:
def wgtMean(price,volume,window):
    return ( price * volume ).rolling(window).sum() / volume.rolling(window).sum()

In [4]:
def macd(price,volume,window1,window2):
    return wgtMean(price,volume,window1) - wgtMean(price,volume,window2)

## Signal functions

Sig1 is MACD crossover. Sig2 is MACD crossing over its moving average.

In [5]:
# Use dictionary as argument.
def sig1(macd,window):
    return macd > 0
    
def sig2(macd,window):
    return ( macd - macd.rolling(window).mean() ) > 0

### Compute Value
Start with value 1. Buy when signal > 0. Sell when signal < 0. Compounded.

In [6]:
def getValues(price,signal):
    value = 1.0
    values = []
    dates = []
    paid = False
    for i, sig in signal.iteritems():
        if sig and not paid:
            paid = price[i]
        elif not sig and paid:
            value *= price[i] / paid
            paid = False
        values.append(value)
        dates.append(i)
    return pd.Series(values,index=dates)

In [7]:
def countTrades(values):
    trades=0
    for i in range(len(values) - 1):
        if i != values[i+1]: trades += 1
    return trades

# Default variables

In [8]:
periods = [900,1800,7200] # 15mins, 30mins, 2hours
pairs = [
    'USDT_BCH',
    'USDT_BTC',
    'USDT_DASH',
    'USDT_ETC',
    'USDT_ETH',
    'USDT_LTC',
    'USDT_NXT',
    'USDT_REP',
    'USDT_STR',
    'USDT_XMR',
    'USDT_XRP',
    'USDT_ZEC'
]

pair='USDT_BTC'
period=900 # 15 mins
length=3600 # 900 hours
window1=3 # 45 mins
window2=60 # 15 hours
end = time.time() # Now
start = end - ( length * period )
delta=2
signal=sig1
signals=[sig1,sig2]

# Heat map
Heat map of the final values over various X-day versus Y-day signals.

In [9]:
# Needs - Number of trades, Percent strategies beating hold, Better table, Mean/median/stddev/etc, Sharpe, yearly
def getMacdHeat(pair,period,start,end,window1,window2,delta,signal):
    df = loadPolo(pair,period,start,end)
    heat = np.full((window2 - window1 - delta,window2 - window1 - delta),1.0)
    for i in range(window1, window2 - delta):
        for j in range( i + delta, window2):
            df['macd'] = macd(df['price'],df['volume'],i,j)
            values = getValues( df['price'], signal( df['macd'], j)) # Dictionary depending on signal
            heat[j - window1 - delta, i - window1] = values.iloc[len(values) - 1]
    return df,heat

# Table of max/min values

# Graph heat map

In [10]:
def macdHeat(pair,period,start,end,window1,window2,delta,signal):
    df,heat = getMacdHeat(pair,period,start,end,window1,window2,delta,signal)
    indices = np.where( heat == np.max( heat[ np.nonzero(heat-1) ] ))
    max={'value':heat[ indices[0][0],indices[1][0] ],
        'window1':indices[1][0]+window1,
        'window2':indices[0][0]+window1+delta
        }
    indices = np.where( heat == np.min( heat[ np.nonzero(heat-1) ] ))
    min={'value': heat[ indices[0][0],indices[1][0] ],
        'window1':indices[1][0]+window1,
        'window2':indices[0][0]+window1+delta
        }
    
    yearly = 365 * 24 * 3600 / ( length * period )
    hold = df.iloc[len(df)-1]['price'] / df.iloc[window2-1]['price']
    base = pair.split('_')[0]
    print("Hold: " + str(np.log( hold )) + ' -> ' + str(hold) + base + ' ~ ' + str(np.power(hold,yearly))  + base + ' yearly')
    print( 'Min @ (' + str(min['window1']) +','+str(min['window2'])+') of ' + str(np.log(min['value'])) + ' -> ' + str( min['value'] ) + base +' ~ ' + str(np.power(min['value'],yearly))  + base + ' yearly' )
    print( 'Max @ (' + str(max['window1']) +','+str(max['window2'])+') of ' + str(np.log(max['value'])) + ' -> ' + str( max['value'] ) + base +' ~ ' + str(np.power(max['value'],yearly))  + base + ' yearly' )

    iplot( [ go.Heatmap(z=np.log(heat),x=[window1,window1 + 1],y=[window1 + delta,window1 + delta + 1]) ] , filename='heat_'+pair)
    
    return heat

## Heat maps

In [13]:
%%time
totalHeat = np.full((window2 - window1 - delta,window2 - window1 - delta),0.0)
for pair in pairs:
    df,heat = getMacdHeat(pair,period,start,end,window1,window2,delta,signal)
    totalHeat += heat
print('Average')
iplot( [ go.Heatmap(z=np.log(totalHeat/len(pairs)),x=[window1,window1 + 1],y=[window1 + delta,window1 + delta + 1]) ] , filename='heat_'+pair)

Average


CPU times: user 4min 29s, sys: 56 ms, total: 4min 29s
Wall time: 4min 33s


In [ ]:
%%time
grandTotalHeat = np.full((window2 - window1 - delta,window2 - window1 - delta),0.0)
for i in range(7):
    print( 'Days back: ' + str(30 * i) )
    epsilon = 3600*24*30*i # 30*i days ago
    totalHeat = np.full((window2 - window1 - delta,window2 - window1 - delta),0.0)
    for pair in pairs:
        df,heat = getMacdHeat(pair,period,start-epsilon,end-epsilon,window1,window2,delta,signal)
        totalHeat += heat
    iplot( [ go.Heatmap(z=np.log(totalHeat/len(pairs)),x=[window1,window1 + 1],y=[window1 + delta,window1 + delta + 1]) ] , filename='heat_'+pair)
    grandTotalHeat += totalHeat
print('Average')
iplot( [ go.Heatmap(z=np.log(grandTotalHeat/7),x=[window1,window1 + 1],y=[window1 + delta,window1 + delta + 1]) ] , filename='heat_'+pair)

Days back: 0
